<a href="https://colab.research.google.com/github/whkwls2653/Emotion-Recognition/blob/main/cogmen_feature_foramt19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torc
##Kobert pretrained model
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
##numpy version for error
!pip install numpy==1.23.1

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
print(np.__version__)
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook

1.23.1


In [ ]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [4]:
import csv
import pandas as pd
import os
import pickle
from tqdm import tqdm
annotation_root= '/content/gdrive/MyDrive/emotion_recognition/dataset/KEMDy19/annotation'
txt_label_zip_toteval_root='/content/gdrive/MyDrive/emotion_recognition/dataset/2019cogmen_format.pkl'
ambig_annotation=0
no_file=0
## formating features dim 9 lists
sess_ids=[]
sess_speakers=dict()
sess_labels=dict()
sess_text=dict()
sess_audio=dict()
sess_visual=dict()
sess_sentence=dict()
sess_train=[]
sess_test=[]
if os.path.isfile(txt_label_zip_toteval_root):
  with open(txt_label_zip_toteval_root,'rb') as f:
    allfile_datalist=pickle.load(f)
else:
  allfile_datalist=[]
  for i, annotation in tqdm(enumerate(os.listdir(annotation_root))):
    all_txts=[]
    # print('annotaion file : (%d / %d)'%(i,len(os.listdir(annotation_root))))
    session_file = pd.read_csv(os.path.join(annotation_root,annotation)) 
    # print(session_file) 
    segments=session_file['Segment ID']
    # print(segments)
    
    for j in range(1,len(segments)):

      #index starts from 1
      f_name=segments[j]
      
      # print("f_name:",f_name)
      sess_num=f_name.split('_')[0][-2:]
      sc_pro_num=f_name.split('_')[0]+'_'+f_name.split('_')[1]
      # print(sess_num,sc_pro_num)
      
    #  /content/gdrive/MyDrive/감정인식 대회/데이터셋/KEMDy19/wav/Session01/Sess01_script01/Sess01_script01_F001.txt
      file_loc='/content/gdrive/MyDrive/emotion_recognition/dataset/KEMDy19/wav/Session%s/%s/%s.txt'%(sess_num,sc_pro_num,f_name)
      # print(file_loc)
      if not os.path.isfile(file_loc):
        print("no file location :",file_loc)
        no_file+=1
        continue
      with open(file_loc) as f:
        lines = f.readlines()
        all_txts.append(lines)
      # print(all_txts) 


    session_file.loc[(session_file['Total Evaluation'] == "fear"), 'Total Evaluation'] = 0  #공포 => 0
    session_file.loc[(session_file['Total Evaluation'] == "surprise"), 'Total Evaluation'] = 1  #놀람 => 1
    session_file.loc[(session_file['Total Evaluation'] == "angry"), 'Total Evaluation'] = 2  #분노 => 2
    session_file.loc[(session_file['Total Evaluation'] == "sad"), 'Total Evaluation'] = 3  #슬픔 => 3
    session_file.loc[(session_file['Total Evaluation'] == "neutral"), 'Total Evaluation'] = 4  #중립 => 4
    session_file.loc[(session_file['Total Evaluation'] == "happy"), 'Total Evaluation'] = 5  #행복 => 5
    session_file.loc[(session_file['Total Evaluation'] == "disgust"), 'Total Evaluation'] = 6  #혐오 => 6



    
    # print(len(all_txts))
    ## zip txts and labels,  session_file start from 1
    if not (len(all_txts)==len(session_file['Total Evaluation'][1:])):
      print("txt_session length not equal at :",annotation,len(all_txts),len(session_file['Total Evaluation'][1:]))
    # print(len(session_file['Total Evaluation'][1:]))
    for q, label in zip(all_txts, session_file['Total Evaluation'][1:])  :
      if type(label) is not int :
        ambig_annotation+=1
        continue
      words=q[0]
      if '\n' in words or'/'in words :
        new_words=words.replace('\n','')
        new_words=new_words.replace('o/','')
        new_words=new_words.replace('I/','')
        new_words=new_words.replace('b/','')
        new_words=new_words.replace('u/','')
        new_words=new_words.replace('l/','')
        new_words=new_words.replace('n/','')
        new_words=new_words.replace('N/','')
        new_words=new_words.replace('s/','')
        new_words=new_words.replace('c/','')
        new_words=new_words.replace('(','')
        new_words=new_words.replace(')','')
        new_words=new_words.replace('+',' ')
        words=new_words
      data = []
      data.append(words)
      # print('Q',type(q),q[0])
      # print('label',label)
      data.append(str(label))
      allfile_datalist.append(data)
    print(allfile_datalist)
    # allfile_datalist.append(data_list)



  with open(txt_label_zip_toteval_root,'wb') as f:
    pickle.dump(allfile_datalist,f)
# print("data_list length :",len(data_list))
# print("sample : ",data_list[0])
  
# print(len(all_txts))





1it [02:17, 137.89s/it]

True
[['어 저 지그 지금 사람 친 거야? 지금 사람 친 거 맞지? 그치?', '1'], [' 몰라.   아 몰라 어떡해. ', '0'], ['아이 씨 그러니까 나 말렸어야지. 술 먹어서 운전 안 한다고 했잖아. ', '0'], ['아이 씨 괜히 운전해서 이씨.', '0'], [' 지섭씨. 일단 112에 신고하자.', '0'], ['잠깐만. 잠깐만 기다려봐. 이 그러니 그러니까 그 좀 생각좀  해보자고 그. ', '0'], ['아 지금 생각할게 할게 뭐가 있어.  지섭씨는 여기있어. 일단 내가 가서 보고  올테니까.', '0'], ['  죽었으면  어떡하지? 나 수 술도  술도 마셨는데  신고를 해야되나? 아이 그니까 이게  나중에 더 일이  커질건데 그면 어떡하지. 그니까 내가 그니까 이거. ', '0'], ['이 야밤에 돌아다니다가 왜 내 차에 치이고 난리야 이씨.  ', '0'], ['지섭씨.', '2'], ['아 아니 그게 말야. 아무것도 도로에 아무것도 없어.', '0'], ['어? 아무것도 없다니? 그게 말이 돼? 아 틀림없이 사람을 쳤잖아. 너도 봤잖아. ', '0'], ['아 그래 근데 지금 없어. 죽었는지 살았는지 우리가 친 사람이 없단 말야.', '0'], [' 그 그럼 뭐지? 어? 뭐 뭐.', '0'], [' 아  아 그래 맞아. 고라니였나보다. 그치? 고라니였어.  그니까 너랑 나랑 술을 마셔서 고라니를 이 사람으로 착각을 한 거야. 그지?', '0'], ['  지섭씨 사람하고 고라니를 구분 못 할 정돈 아니잖아', '4'], ['하 그러니까.  ', '0'], ['잠깐만. 잠깐만 우 우리 어 냉정하게 생각해보자.  어차피 사람이 인지 아닌지도 모르는데 그럼 내가 그냥 가도 된다는 거지? 그럼 된다는 거 아냐?', '0'], ['지금 무슨 소릴 하는 거야. 지금 어딘가에 있을 거야. 우리가 사람을 쳤는데 당연히 찾아서 병원에 데려가야지. ', '0'], [' 우리 술마셨잖아.', '0'], ['그러니까 이 그냥 이대로 가자? 어? 나

2it [05:22, 165.17s/it]

True
[['어 저 지그 지금 사람 친 거야? 지금 사람 친 거 맞지? 그치?', '1'], [' 몰라.   아 몰라 어떡해. ', '0'], ['아이 씨 그러니까 나 말렸어야지. 술 먹어서 운전 안 한다고 했잖아. ', '0'], ['아이 씨 괜히 운전해서 이씨.', '0'], [' 지섭씨. 일단 112에 신고하자.', '0'], ['잠깐만. 잠깐만 기다려봐. 이 그러니 그러니까 그 좀 생각좀  해보자고 그. ', '0'], ['아 지금 생각할게 할게 뭐가 있어.  지섭씨는 여기있어. 일단 내가 가서 보고  올테니까.', '0'], ['  죽었으면  어떡하지? 나 수 술도  술도 마셨는데  신고를 해야되나? 아이 그니까 이게  나중에 더 일이  커질건데 그면 어떡하지. 그니까 내가 그니까 이거. ', '0'], ['이 야밤에 돌아다니다가 왜 내 차에 치이고 난리야 이씨.  ', '0'], ['지섭씨.', '2'], ['아 아니 그게 말야. 아무것도 도로에 아무것도 없어.', '0'], ['어? 아무것도 없다니? 그게 말이 돼? 아 틀림없이 사람을 쳤잖아. 너도 봤잖아. ', '0'], ['아 그래 근데 지금 없어. 죽었는지 살았는지 우리가 친 사람이 없단 말야.', '0'], [' 그 그럼 뭐지? 어? 뭐 뭐.', '0'], [' 아  아 그래 맞아. 고라니였나보다. 그치? 고라니였어.  그니까 너랑 나랑 술을 마셔서 고라니를 이 사람으로 착각을 한 거야. 그지?', '0'], ['  지섭씨 사람하고 고라니를 구분 못 할 정돈 아니잖아', '4'], ['하 그러니까.  ', '0'], ['잠깐만. 잠깐만 우 우리 어 냉정하게 생각해보자.  어차피 사람이 인지 아닌지도 모르는데 그럼 내가 그냥 가도 된다는 거지? 그럼 된다는 거 아냐?', '0'], ['지금 무슨 소릴 하는 거야. 지금 어딘가에 있을 거야. 우리가 사람을 쳤는데 당연히 찾아서 병원에 데려가야지. ', '0'], [' 우리 술마셨잖아.', '0'], ['그러니까 이 그냥 이대로 가자? 어? 나

3it [05:23, 90.25s/it] 

...

True
[['어 저 지그 지금 사람 친 거야? 지금 사람 친 거 맞지? 그치?', '1'], [' 몰라.   아 몰라 어떡해. ', '0'], ['아이 씨 그러니까 나 말렸어야지. 술 먹어서 운전 안 한다고 했잖아. ', '0'], ['아이 씨 괜히 운전해서 이씨.', '0'], [' 지섭씨. 일단 112에 신고하자.', '0'], ['잠깐만. 잠깐만 기다려봐. 이 그러니 그러니까 그 좀 생각좀  해보자고 그. ', '0'], ['아 지금 생각할게 할게 뭐가 있어.  지섭씨는 여기있어. 일단 내가 가서 보고  올테니까.', '0'], ['  죽었으면  어떡하지? 나 수 술도  술도 마셨는데  신고를 해야되나? 아이 그니까 이게  나중에 더 일이  커질건데 그면 어떡하지. 그니까 내가 그니까 이거. ', '0'], ['이 야밤에 돌아다니다가 왜 내 차에 치이고 난리야 이씨.  ', '0'], ['지섭씨.', '2'], ['아 아니 그게 말야. 아무것도 도로에 아무것도 없어.', '0'], ['어? 아무것도 없다니? 그게 말이 돼? 아 틀림없이 사람을 쳤잖아. 너도 봤잖아. ', '0'], ['아 그래 근데 지금 없어. 죽었는지 살았는지 우리가 친 사람이 없단 말야.', '0'], [' 그 그럼 뭐지? 어? 뭐 뭐.', '0'], [' 아  아 그래 맞아. 고라니였나보다. 그치? 고라니였어.  그니까 너랑 나랑 술을 마셔서 고라니를 이 사람으로 착각을 한 거야. 그지?', '0'], ['  지섭씨 사람하고 고라니를 구분 못 할 정돈 아니잖아', '4'], ['하 그러니까.  ', '0'], ['잠깐만. 잠깐만 우 우리 어 냉정하게 생각해보자.  어차피 사람이 인지 아닌지도 모르는데 그럼 내가 그냥 가도 된다는 거지? 그럼 된다는 거 아냐?', '0'], ['지금 무슨 소릴 하는 거야. 지금 어딘가에 있을 거야. 우리가 사람을 쳤는데 당연히 찾아서 병원에 데려가야지. ', '0'], [' 우리 술마셨잖아.', '0'], ['그러니까 이 그냥 이대로 가자? 어? 나

5it [08:15, 78.63s/it] 

True
[['어 저 지그 지금 사람 친 거야? 지금 사람 친 거 맞지? 그치?', '1'], [' 몰라.   아 몰라 어떡해. ', '0'], ['아이 씨 그러니까 나 말렸어야지. 술 먹어서 운전 안 한다고 했잖아. ', '0'], ['아이 씨 괜히 운전해서 이씨.', '0'], [' 지섭씨. 일단 112에 신고하자.', '0'], ['잠깐만. 잠깐만 기다려봐. 이 그러니 그러니까 그 좀 생각좀  해보자고 그. ', '0'], ['아 지금 생각할게 할게 뭐가 있어.  지섭씨는 여기있어. 일단 내가 가서 보고  올테니까.', '0'], ['  죽었으면  어떡하지? 나 수 술도  술도 마셨는데  신고를 해야되나? 아이 그니까 이게  나중에 더 일이  커질건데 그면 어떡하지. 그니까 내가 그니까 이거. ', '0'], ['이 야밤에 돌아다니다가 왜 내 차에 치이고 난리야 이씨.  ', '0'], ['지섭씨.', '2'], ['아 아니 그게 말야. 아무것도 도로에 아무것도 없어.', '0'], ['어? 아무것도 없다니? 그게 말이 돼? 아 틀림없이 사람을 쳤잖아. 너도 봤잖아. ', '0'], ['아 그래 근데 지금 없어. 죽었는지 살았는지 우리가 친 사람이 없단 말야.', '0'], [' 그 그럼 뭐지? 어? 뭐 뭐.', '0'], [' 아  아 그래 맞아. 고라니였나보다. 그치? 고라니였어.  그니까 너랑 나랑 술을 마셔서 고라니를 이 사람으로 착각을 한 거야. 그지?', '0'], ['  지섭씨 사람하고 고라니를 구분 못 할 정돈 아니잖아', '4'], ['하 그러니까.  ', '0'], ['잠깐만. 잠깐만 우 우리 어 냉정하게 생각해보자.  어차피 사람이 인지 아닌지도 모르는데 그럼 내가 그냥 가도 된다는 거지? 그럼 된다는 거 아냐?', '0'], ['지금 무슨 소릴 하는 거야. 지금 어딘가에 있을 거야. 우리가 사람을 쳤는데 당연히 찾아서 병원에 데려가야지. ', '0'], [' 우리 술마셨잖아.', '0'], ['그러니까 이 그냥 이대로 가자? 어? 나

5it [10:00, 120.16s/it]


KeyboardInterrupt: ignored

In [ ]:
print(allfile_datalist)
print(len(allfile_datalist))

In [ ]:
#### test1 \n 등빼기
if os.path.isfile(txt_label_zip_toteval_root):
  with open(txt_label_zip_toteval_root,'rb') as f:
    allfile_datalist_processed=pickle.load(f)
for words in allfile_datalist_processed: 
  
  
  if '\n' in words[0] or'/'in words[0] :
    new_words=words[0].replace('\n','')
    new_words=new_words.replace('o/','')
    new_words=new_words.replace('I/','')
    new_words=new_words.replace('b/','')
    new_words=new_words.replace('u/','')
    new_words=new_words.replace('l/','')
    new_words=new_words.replace('n/','')
    new_words=new_words.replace('N/','')
    new_words=new_words.replace('s/','')
    new_words=new_words.replace('(','')
    new_words=new_words.replace(')','')
    new_words=new_words.replace('+',' ')
    words[0]=new_words
    # print(new_words) 
with open(txt_label_zip_toteval_root+'_processed','wb') as f:
  pickle.dump(allfile_datalist_processed,f)

In [ ]:
# if os.path.isfile(txt_label_zip_toteval_root):
with open(txt_label_zip_toteval_root+'_processed','rb') as f:
  allfile_datalist_processed=pickle.load(f)
  print(allfile_datalist_processed)

In [ ]:
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(allfile_datalist_processed, test_size=0.25, random_state=0)
print(len(dataset_train))
print(len(dataset_test))

14442
4815


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 15
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5)#.to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [ ]:
##train
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train().to(device)
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

torch.save({
            'epoch': num_epochs,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, '/content/gdrive/MyDrive/감정인식 대회/데이터셋/2019_processed_15epoch')

<ipython-input-101-752f5961bdb8>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/226 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


epoch 1 batch id 1 loss 0.7593756914138794 train acc 0.671875


In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("공포가")
            elif np.argmax(logits) == 1:
                test_eval.append("놀람이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("중립이")
            elif np.argmax(logits) == 5:
                test_eval.append("행복이")
            elif np.argmax(logits) == 6:
                test_eval.append("혐오가")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
하고싶은 말을 입력해주세요 : 라면좋아
>> 입력하신 내용에서 중립이 느껴집니다.


하고싶은 말을 입력해주세요 : 라면 먹으니 행복해
>> 입력하신 내용에서 중립이 느껴집니다.


하고싶은 말을 입력해주세요 : 화가나네
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 역겨워
>> 입력하신 내용에서 중립이 느껴집니다.


하고싶은 말을 입력해주세요 : 어두운게 무서워
>> 입력하신 내용에서 공포가 느껴집니다.


하고싶은 말을 입력해주세요 : 어제 헤어졌어
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 주식이 많이 올랐어
>> 입력하신 내용에서 중립이 느껴집니다.


하고싶은 말을 입력해주세요 : 날씨가 너무 맑다!
>> 입력하신 내용에서 중립이 느껴집니다.




KeyboardInterrupt: ignored